# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
#pd.set_option('display.max_columns', None)
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.probability import FreqDist
import nltk
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
columns = ['target','id','date','flag','user','text']
data = pd.read_csv('training.1600000.processed.noemoticon.csv', 
                   header=None, names=columns, encoding='ISO-8859-1', 
                   index_col=False)

In [3]:
data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
# Extracting a sample of 10000
subset = data.sample(10000).copy()

In [5]:
display(subset.head(),subset.shape)

,target,id,date,flag,user,text
41191,0,1573986704,Tue Apr 21 03:46:37 PDT 2009,NO_QUERY,daisygrl2277,off to school Damn marlins what the heck happ...
845897,4,1564299956,Mon Apr 20 02:48:25 PDT 2009,NO_QUERY,Ruxxx,I'm new here! So I just want to say HY! Have ...
54154,0,1680948524,Sat May 02 12:52:00 PDT 2009,NO_QUERY,gomer43,@Car1ssa but I'll get fiiiiiiired.
1251225,4,1996459978,Mon Jun 01 15:07:14 PDT 2009,NO_QUERY,magicswebpage,shower was great but now out of bars of soap s...
1275191,4,2001055928,Mon Jun 01 23:42:48 PDT 2009,NO_QUERY,niinio,is getting used to the 4-days-work-3-days-off-...


(10000, 6)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [6]:
def clean_up(s):
    S = str(s)
    S = re.sub(r"http\S+", "", s)
    S = re.sub(r"[^a-zA-Z]+", " ", S).strip().lower()
    return S

def tokenize(s):
    S = str(s)
    return word_tokenize(S)

def stem_and_lemmatize(l):
    ps = PorterStemmer()
    stemmed = [ps.stem(w) for w in l]
    
    lm = WordNetLemmatizer() 
    stemlem = [lm.lemmatize(w) for w in stemmed]
    
    return stemlem

def remove_stopwords(l):
    without_sw = [word for word in l if not word in stopwords.words('english')]
    return without_sw

In [7]:
subset['clean_up'] = subset['text'].apply(clean_up)
subset['tokenize'] = subset['clean_up'].apply(tokenize)
subset['stem_and_lemmatize'] = subset['tokenize'].apply(stem_and_lemmatize)
subset['text_processed'] = subset['stem_and_lemmatize'].apply(remove_stopwords)

In [8]:
subset = subset.drop(['clean_up', 'tokenize', 'stem_and_lemmatize'], axis=1) 

In [9]:
subset.head(5)

,target,id,date,flag,user,text,text_processed
41191,0,1573986704,Tue Apr 21 03:46:37 PDT 2009,NO_QUERY,daisygrl2277,off to school Damn marlins what the heck happ...,"[school, damn, marlin, heck, happen, last, night]"
845897,4,1564299956,Mon Apr 20 02:48:25 PDT 2009,NO_QUERY,Ruxxx,I'm new here! So I just want to say HY! Have ...,"[new, want, say, hy, nice, day, guy, xoxo]"
54154,0,1680948524,Sat May 02 12:52:00 PDT 2009,NO_QUERY,gomer43,@Car1ssa but I'll get fiiiiiiired.,"[car, ssa, get, fiiiiiiir]"
1251225,4,1996459978,Mon Jun 01 15:07:14 PDT 2009,NO_QUERY,magicswebpage,shower was great but now out of bars of soap s...,"[shower, wa, great, bar, soap, sorri, guy, wa,..."
1275191,4,2001055928,Mon Jun 01 23:42:48 PDT 2009,NO_QUERY,niinio,is getting used to the 4-days-work-3-days-off-...,"[get, use, day, work, day, rhythm, feel, right]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [10]:
# Put all this cleaning together

def re_blob(row):
    return " ".join(row['text_processed'])

subset['clean_blob'] = subset.apply(re_blob,axis=1)

In [11]:
subset.head(5)

,target,id,date,flag,user,text,text_processed,clean_blob
41191,0,1573986704,Tue Apr 21 03:46:37 PDT 2009,NO_QUERY,daisygrl2277,off to school Damn marlins what the heck happ...,"[school, damn, marlin, heck, happen, last, night]",school damn marlin heck happen last night
845897,4,1564299956,Mon Apr 20 02:48:25 PDT 2009,NO_QUERY,Ruxxx,I'm new here! So I just want to say HY! Have ...,"[new, want, say, hy, nice, day, guy, xoxo]",new want say hy nice day guy xoxo
54154,0,1680948524,Sat May 02 12:52:00 PDT 2009,NO_QUERY,gomer43,@Car1ssa but I'll get fiiiiiiired.,"[car, ssa, get, fiiiiiiir]",car ssa get fiiiiiiir
1251225,4,1996459978,Mon Jun 01 15:07:14 PDT 2009,NO_QUERY,magicswebpage,shower was great but now out of bars of soap s...,"[shower, wa, great, bar, soap, sorri, guy, wa,...",shower wa great bar soap sorri guy wa befor to...
1275191,4,2001055928,Mon Jun 01 23:42:48 PDT 2009,NO_QUERY,niinio,is getting used to the 4-days-work-3-days-off-...,"[get, use, day, work, day, rhythm, feel, right]",get use day work day rhythm feel right


In [12]:
# Merge lists in clean_blob

merged_list = subset['clean_blob'].tolist()
len(merged_list)

10000

In [13]:
# Create one list of all words

split_list = []
all_words = []

for words in merged_list:
    splitwordslist = words.split()
    split_list.append(splitwordslist)
    
for lists in split_list:
    for word in lists:
        all_words.append(word)

In [14]:
# Get frequency distribution of words

freq = FreqDist(all_words)

In [15]:
# Get top5000 common words 

topwords = freq.most_common(5000)
print(len(topwords))

5000


In [16]:
#topwords

In [17]:
# Get just words from frequency list

top5000 = [words for words,frequency in topwords]
print(len(top5000))

5000


In [18]:
#top5000

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [19]:
def find_features(doc):
    words = set(doc)
    features = {}
    for w in top5000:
        features[w] = (w in words)
    return features

In [20]:
# Get feature set

featuresets = [(find_features(tweet), target) for (tweet, target) in list(zip(subset['text_processed'], subset['target']))]


In [21]:
len(featuresets)

10000

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [26]:
# set that we'll train our classifier with
train_set = featuresets[:1900]

# set that we'll test against.
test_set = featuresets[1900:]

In [27]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [28]:
nltk.classify.accuracy(classifier, test_set)

0.6793827160493827

In [29]:
classifier.show_most_informative_features(15)

Most Informative Features
                     sad = True                0 : 4      =     28.5 : 1.0
                    damn = True                0 : 4      =     10.2 : 1.0
                    rain = True                0 : 4      =      9.5 : 1.0
                    poor = True                0 : 4      =      8.8 : 1.0
                    suck = True                0 : 4      =      8.2 : 1.0
                     cri = True                0 : 4      =      8.1 : 1.0
                    sick = True                0 : 4      =      8.1 : 1.0
                   thank = True                4 : 0      =      7.5 : 1.0
                    cute = True                4 : 0      =      7.3 : 1.0
                    late = True                0 : 4      =      6.7 : 1.0
                  believ = True                0 : 4      =      6.0 : 1.0
                   found = True                0 : 4      =      6.0 : 1.0
                 serious = True                0 : 4      =      6.0 : 1.0

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [30]:
# Increase training set
train_set = featuresets[:3000]

# Test set
test_set = featuresets[1900:]

classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.717037037037037

By increasing our training set, model performance improved to 0.71!

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here